<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/bra1/lightcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install catboost

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool
from sklearn.isotonic import IsotonicRegression

from sklearn.model_selection import KFold


# warningsを非表示にする
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_train_allnum.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_test_allnum.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [5]:
df_train.head()

,id,Age,TypeofContact,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,Passport,MonthlyIncome,...,PitchSatisfactionScore_5,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚
0,0,1.145527e+00,1,0.077180,0,-1.908669,0.361786,1.038068,1,-1.352770,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,1.766003e+00,0,-0.067460,0,-1.908669,0.361786,-0.699052,1,0.639729,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,2,7.347917e-16,1,-0.646021,1,-1.908669,-0.746633,0.459028,0,-1.032001,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,-1.988390e-01,1,0.511101,1,-1.908669,-0.746633,-1.278092,0,-0.388082,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,4,9.387011e-01,0,0.366461,1,-1.908669,-0.746633,0.459028,0,-1.292825,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
df_train.drop(["id"], axis=1, inplace=True)
df_test.drop(["id"], axis=1, inplace=True)

In [7]:
df_test.head()

,Age,TypeofContact,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,Passport,MonthlyIncome,ProdTaken,...,PitchSatisfactionScore_5,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚
0,0.938701,1,-0.212100,0,-1.908669,0.361786,2.196148,0,1.863454,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.922728,1,-0.356740,1,-1.908669,0.361786,0.459028,1,-0.742793,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,-1.439792,1,-0.790661,1,-1.908669,0.361786,-1.278092,0,-1.272115,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-1.853443,0,-1.079941,0,-1.908669,0.361786,-1.278092,0,-1.273769,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.214812,0,-1.079941,0,-1.908669,0.361786,-1.278092,0,-1.155267,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [8]:
target = "ProdTaken"
train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [9]:
test_x = df_test.drop(target, axis=1)

# Lightgbmモデル


In [10]:
params_lgb = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 25,
    'learning_rate': 0.05,
    'feature_fraction': 0.4,
    'bagging_fraction': 0.8,
    'bagging_freq': 3,
    'verbose': -1,
    'lambda_l1': 0.81,
    'lambda_l2': 90
}

In [11]:
lgbr =lgb.LGBMClassifier(**params_lgb)

In [12]:
auc_sum = []
preds = []
splits = 7
kf2 = KFold(n_splits = splits, shuffle = True, random_state = 0)
for fold, (train_idx, valid_idx) in enumerate(kf2.split(train_x)):
    X_tr, y_tr = train_x.iloc[train_idx, :], train_y.iloc[train_idx]
    X_val, y_val = train_x.iloc[valid_idx, :], train_y.iloc[valid_idx]

    # LightGBM モデルの設定
    model = lgb.LGBMClassifier(
        **params_lgb  # 'n_estimators' を含むすべてのパラメータを渡す
    )

    # モデルのトレーニング
    model.fit(
        X_tr, y_tr,
        eval_metric='auc',
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(100, verbose=True)]
    )

    #y_pred = model.predict(X_val)[:, 1]
    preds_y_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, preds_y_proba)
    pred = model.predict_proba(test_x)[:, 1]
    preds.append(pred)

    print(f'Fold {fold + 1}: AUC = {auc:4f}')
    auc_sum.append(auc)

# 特徴量の重要度を出力する
importances = model.feature_importances_
column_names = train_x.columns.values
importance_list = [
    [importances[i], column_names[i]] for i in range(len(column_names))
]
importance_list.sort(reverse=True, key=lambda x: x[0])

print("Feature importances:")
for importance, feature in importance_list:
    print(f"{importance:.4f} : {feature}")

print(np.mean(auc_sum))

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.829936
Fold 1: AUC = 0.829936
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[96]	valid_0's auc: 0.883375
Fold 2: AUC = 0.883375
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[97]	valid_0's auc: 0.848697
Fold 3: AUC = 0.848697
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.780403
Fold 4: AUC = 0.780403
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[96]	valid_0's auc: 0.867738
Fold 5: AUC = 0.867738
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.818796
Fold 6: AUC = 0.818796
Training until validation

In [13]:
last_pred_1 = 3489 * [0.0]

for i in range(3489):
    for j in range(splits):
        last_pred_1[i] += preds[j][i]
        print(last_pred_1[i])
    last_pred_1[i] /= splits

ensemble_preds = []
ensemble_preds.append(last_pred_1)

ストリーミング出力は最後の 5000 行に切り捨てられました。
1.500799549288069
1.7653116157854813
0.053680041039103295
0.10836283895739975
0.1642091522399011
0.2137392073159087
0.2692726304259973
0.3235600102870296
0.3792111158145943
0.04311905263806354
0.08992079628998437
0.1350732537496413
0.17872350877219284
0.22510074490947932
0.26993934007064013
0.3143262241087198
0.16069844978807465
0.30629470912858014
0.45496855875611675
0.6150583347410055
0.7583397774826011
0.9062318343603222
1.0523881805348336
0.05015301160816271
0.10342943904586227
0.15455518882940356
0.20197118727918006
0.25321455587387526
0.3046206125416806
0.3565584680254018
0.07040964066544975
0.1424558030720018
0.21636455804038165
0.2873502222128346
0.3643605406832656
0.4406633415261805
0.5180736965417734
0.18051661492061447
0.3621053367289354
0.5592090694238164
0.7321234923336866
0.9079034601551803
1.0937153275390037
1.2764951467623924
0.0670543694426209
0.14038040828359225
0.21484803510477385
0.2932764588463251
0.3735148544223802
0.460419165130336

In [14]:
count=0

In [15]:
from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/En_light_{timestamp}_{count:03d}.csv"
ss[1] = last_pred_1
ss.to_csv(file_name, header=False, index=False)

# Catboostモデル

In [16]:
params_cat = {
    "iterations": 3000,  # n_estimatorsに相当
    "learning_rate": 0.01,
    "colsample_bylevel": 0.8,  # colsample_bytreeに相当
    "random_seed": 0,
    "verbose": 100,  # 学習の進捗を100回毎に表示
    "use_best_model": True,  # 早期停止のための設定
}

In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

auc_sum = []
preds = []
splits = 7
kf2 = KFold(n_splits=splits, shuffle=True, random_state=0)

for fold, (train_idx, valid_idx) in enumerate(kf2.split(train_x)):
    X_tr, y_tr = train_x.iloc[train_idx, :], train_y.iloc[train_idx]
    X_val, y_val = train_x.iloc[valid_idx, :], train_y.iloc[valid_idx]

    # CatBoost モデルの設定
    model = CatBoostClassifier(
        **params_cat,  # CatBoostのパラメータをここに渡す
        eval_metric='AUC',
        early_stopping_rounds=100
    )

    # モデルのトレーニング
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        #verbose=True
        verbose_eval=100
    )

    # 予測とAUCの計算
    preds_y_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, preds_y_proba)
    pred = model.predict_proba(test_x)[:, 1]
    preds.append(pred)

    print(f'Fold {fold + 1}: AUC = {auc:4f}')
    auc_sum.append(auc)

# 特徴量の重要度を出力する
importances = model.get_feature_importance()
column_names = train_x.columns.values
importance_list = [
    [importances[i], column_names[i]] for i in range(len(column_names))
]
importance_list.sort(reverse=True, key=lambda x: x[0])

print("Feature importances:")
for importance, feature in importance_list:
    print(f"{importance:.4f} : {feature}")

print(np.mean(auc_sum))

0:	test: 0.6980286	best: 0.6980286 (0)	total: 53.4ms	remaining: 2m 40s
100:	test: 0.8261049	best: 0.8261049 (100)	total: 1.92s	remaining: 55.2s
200:	test: 0.8355485	best: 0.8360890 (190)	total: 3.97s	remaining: 55.3s
300:	test: 0.8369793	best: 0.8375199 (294)	total: 6.51s	remaining: 58.4s
400:	test: 0.8380286	best: 0.8389507 (375)	total: 7.97s	remaining: 51.6s
500:	test: 0.8395548	best: 0.8403816 (489)	total: 9.07s	remaining: 45.3s
600:	test: 0.8413355	best: 0.8416852 (592)	total: 10.3s	remaining: 41.1s
700:	test: 0.8433386	best: 0.8435930 (686)	total: 11.7s	remaining: 38.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8435930048
bestIteration = 686

Shrink model to first 687 iterations.
Fold 1: AUC = 0.843593
0:	test: 0.8397558	best: 0.8397558 (0)	total: 3.39ms	remaining: 10.2s
100:	test: 0.8840332	best: 0.8854153 (98)	total: 1.13s	remaining: 32.5s
200:	test: 0.8948598	best: 0.8948598 (200)	total: 1.84s	remaining: 25.7s
300:	test: 0.8975582	best: 0.8981177 (292

In [18]:
last_pred_2 = 3489 * [0.0]

for i in range(3489):
    for j in range(splits):
        last_pred_2[i] += preds[j][i]
    last_pred_2[i] /= splits

ensemble_preds.append(last_pred_2)

In [19]:
from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/Em_cat_{timestamp}_{count:03d}.csv"
ss[1] = last_pred_2
ss.to_csv(file_name, header=False, index=False)

アンサンブル

In [75]:
def find_permutations(n):
    perm_list = []
    current_perm = [0] * n

    while True:
        if sum(current_perm) == 10:
            perm_list.append(tuple(current_perm))

        i = n-1
        while i >= 0 and current_perm[i] == 10:
            current_perm[i] = 0
            i -= 1

        if i < 0:
            break

        current_perm[i] += 1

    return perm_list

n = 2
permutations = find_permutations(n)
last_pred = len(train_y) * [0.0]
l = []
for i in range(n):
    l.append(0.0)
best_score = 0
for perm in permutations:
    for i in range(len(train_y)):
        last_pred_list = len(train_y) * l
        last_pred_list[i] = [last_pred_1[i], last_pred_2[i]]
        last_pred[i] = np.dot(last_pred_list[i], perm) / 10

   # print(last_pred)
    #auc_test = roc_auc_score(train_x, last_pred)
    #if auc_test > best_score:
     #   best_score = auc_test
     #   best_perm = perm
    #print(perm, auc_test)
print('best param is ', best_perm, ', best score is ', best_score)

ストリーミング出力は最後の 5000 行に切り捨てられました。
1979 (9, 1)
1980 (9, 1)
1981 (9, 1)
1982 (9, 1)
1983 (9, 1)
1984 (9, 1)
1985 (9, 1)
1986 (9, 1)
1987 (9, 1)
1988 (9, 1)
1989 (9, 1)
1990 (9, 1)
1991 (9, 1)
1992 (9, 1)
1993 (9, 1)
1994 (9, 1)
1995 (9, 1)
1996 (9, 1)
1997 (9, 1)
1998 (9, 1)
1999 (9, 1)
2000 (9, 1)
2001 (9, 1)
2002 (9, 1)
2003 (9, 1)
2004 (9, 1)
2005 (9, 1)
2006 (9, 1)
2007 (9, 1)
2008 (9, 1)
2009 (9, 1)
2010 (9, 1)
2011 (9, 1)
2012 (9, 1)
2013 (9, 1)
2014 (9, 1)
2015 (9, 1)
2016 (9, 1)
2017 (9, 1)
2018 (9, 1)
2019 (9, 1)
2020 (9, 1)
2021 (9, 1)
2022 (9, 1)
2023 (9, 1)
2024 (9, 1)
2025 (9, 1)
2026 (9, 1)
2027 (9, 1)
2028 (9, 1)
2029 (9, 1)
2030 (9, 1)
2031 (9, 1)
2032 (9, 1)
2033 (9, 1)
2034 (9, 1)
2035 (9, 1)
2036 (9, 1)
2037 (9, 1)
2038 (9, 1)
2039 (9, 1)
2040 (9, 1)
2041 (9, 1)
2042 (9, 1)
2043 (9, 1)
2044 (9, 1)
2045 (9, 1)
2046 (9, 1)
2047 (9, 1)
2048 (9, 1)
2049 (9, 1)
2050 (9, 1)
2051 (9, 1)
2052 (9, 1)
2053 (9, 1)
2054 (9, 1)
2055 (9, 1)
2056 (9, 1)
2057 (9, 1)
2058 (9, 1)
2059 (9,

In [76]:
 [last_pred_1[1], last_pred_2[1]]

[0.16852593069015756, 0.22664528337745277]

In [ ]:
lastにはいってるのはtestのy
じゃん